In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import joblib

# 读取输入数据
input_file = "input.csv"
df_input = pd.read_csv(input_file)

# 选择输入特征
input_features = ["Time", "AntriebsmomentVA", "AntriebsmomentHA", "Lenkwinkel", "BremsmomentVA", "BremsmomentHA", "Geschw_S"]

# 加载归一化器
scalers = joblib.load("scalers.pkl")

# 对输入数据进行归一化
for feature in input_features:
    df_input[feature] = scalers[feature].transform(df_input[[feature]])

def create_sequences(X, time_steps=10):
    Xs = []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
    return np.array(Xs)

# 设定时间步长
time_steps = 10
X_input = create_sequences(df_input[input_features].values, time_steps)

# 加载模型
model1 = load_model("model_giergeschwindigkeit.keras")
model2 = load_model("model_gierwinkel.keras")
model3 = load_model("model_geschw.keras")
model4 = load_model("model_ks.keras")

# 第一阶段：预测 Giergeschwindigkeit
giergeschwindigkeit_pred = model1.predict(X_input)

# 归一化反变换
giergeschwindigkeit_pred = scalers["Giergeschwindigkeit"].inverse_transform(giergeschwindigkeit_pred)

# 第二阶段：预测 Gierwinkel
X2 = np.hstack([df_input[["Time", "Lenkwinkel"]].values[len(df_input) - len(giergeschwindigkeit_pred):], giergeschwindigkeit_pred])
X2 = X2.reshape(X2.shape[0], 1, X2.shape[1])
gierwinkel_pred = model2.predict(X2)
gierwinkel_pred = scalers["Gierwinkel"].inverse_transform(gierwinkel_pred)

# 第三阶段：预测 Geschw_X, Geschw_Y
X3 = np.hstack([df_input[["Time", "Lenkwinkel", "Geschw_S"]].values[len(df_input) - len(gierwinkel_pred):], gierwinkel_pred])
X3 = X3.reshape(X3.shape[0], 1, X3.shape[1])
geschw_pred = model3.predict(X3)
geschw_pred = scalers["Geschw_X"].inverse_transform(geschw_pred)  # 反归一化 Geschw_X
geschw_pred[:, 1] = scalers["Geschw_Y"].inverse_transform(geschw_pred[:, 1].reshape(-1, 1)).flatten()  # 反归一化 Geschw_Y

# 第四阶段：预测 KS_X, KS_Y
X4 = np.hstack([df_input[["Time", "Lenkwinkel"]].values[len(df_input) - len(geschw_pred):], geschw_pred])
X4 = X4.reshape(X4.shape[0], 1, X4.shape[1])
ks_pred = model4.predict(X4)
ks_pred = scalers["KS_X"].inverse_transform(ks_pred)  # 反归一化 KS_X
ks_pred[:, 1] = scalers["KS_Y"].inverse_transform(ks_pred[:, 1].reshape(-1, 1)).flatten()  # 反归一化 KS_Y

# 组织输出数据
output_df = pd.DataFrame({
    "KS_X": ks_pred[:, 0],
    "KS_Y": ks_pred[:, 1],
    "Giergeschwindigkeit": giergeschwindigkeit_pred[:, 0],
    "Gierwinkel": gierwinkel_pred[:, 0],
    "Geschw_X": geschw_pred[:, 0],
    "Geschw_Y": geschw_pred[:, 1]
})

# 保存预测结果
output_df.to_csv("predictions.csv", index=False)

print("预测完成，结果已保存到 predictions.csv！")


781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 856us/step
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 825us/step
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 865us/step
预测完成，结果已保存到 predictions.csv！
